In [2]:
from cell_segmentation.datasets.pannuke import PanNukeDataset
from torch.utils.data import DataLoader
from tqdm import tqdm

In [27]:
dataset = PanNukeDataset(
    dataset_path="/home/jovyan/cellvit-datasets/PanNuke", 
    folds=[0],
    stardist=False
)

In [4]:
dataset_1 = PanNukeDataset(
    dataset_path="/home/jovyan/cellvit-datasets/PanNuke", 
    folds=[0],
)

In [5]:
image, mask, tissue_type, name = dataset_1.__getitem__(0)

In [6]:
dist_map = PanNukeDataset.gen_distance_prob_maps(mask["instance_map"].detach().cpu().numpy())

In [7]:
dist_map = PanNukeDataset.gen_stardist_maps(mask["instance_map"].detach().cpu().numpy())

100%|██████████| 256/256 [03:29<00:00,  1.22it/s]


In [25]:
dl_stardist = DataLoader(
    dataset=dataset,
    batch_size=1,
    num_workers=12
)

In [19]:
dl = DataLoader(
    dataset=dataset_1,
    batch_size=1,
    num_workers=12
)

In [26]:
for batch in tqdm(dl_stardist, total=len(dl_stardist)):
    batch

  0%|          | 0/2656 [02:27<?, ?it/s]


KeyboardInterrupt: 

In [21]:
for batch in tqdm(dl, total=len(dl_stardist)):
    batch

100%|██████████| 166/166 [00:08<00:00, 20.37it/s]
